In [2]:
import pandas as pd

In [3]:
mag = pd.read_csv('/Users/arup/Documents/ISB/mag_final.csv')
mag

,entity_id,processed_title
0,566794768,29p-aps-61 dy fe_ co_x _2の低温x線回折
1,2367711586,improv program evalu review techniqu base part...
2,2090358667,gorbachev gener secretari becom social democrat
3,2827751863,speed control dc motor
4,596046335,shakespear 's histori
...,...,...
16249,2062546659,coastal fring habitat threaten global warm
16250,2617488891,brasiguaio uma identidad na fronteira brasil/p...
16251,2772711505,activ torqu vector wheel drive fsae electr car
16252,2490723897,technic progress evolut


In [4]:
patent = pd.read_csv('/Users/arup/Documents/ISB/patent_final.csv')
patent

,patent_id,patent_title,uspc_mainclass_title,uspc_subclass_title,wipo_sector_title,wipo_field_title,processed_abstract
0,3969244,Method of adsorbing heavy metals,NaN,NaN,Chemistry,"Macromolecular chemistry, polymers",particul polycondens thiourea thiosemicarbazid...
1,5386430,Excimer laser processing method and apparatus,COHERENT LIGHT GENERATORS,Excimer or exciplex,Electrical engineering,Audio-visual technology,excim laser ablat process form via hole resin ...
2,6710441,"POWER SEMICONDUCTOR SWITCHING DEVICES, POWER C...","ACTIVE SOLID-STATE DEVICES (E.G., TRANSISTORS,...","All contacts on same surface (e.g., lateral st...",Electrical engineering,Semiconductors,power semiconductor switch devic power convert...
3,5441784,Paper base wallcoverings,STOCK MATERIAL OR MISCELLANEOUS ARTICLES,WALL AND SHELF COVERING,Mechanical engineering,Textile and paper machines,invent provid paper base wallcov option decor ...
4,6579624,Functional film having optical and electrical ...,STOCK MATERIAL OR MISCELLANEOUS ARTICLES,THREE DIMENSION IMITATION OR 'TREATED' NATURAL...,Electrical engineering,Audio-visual technology,function film includ transit layer first const...
...,...,...,...,...,...,...,...
39310,10121211,Waste analysis system and method,NaN,NaN,Electrical engineering,IT methods for management,embodi present invent provid techniqu identifi...
39311,6911447,Melanocortin receptor ligands,ORGANIC COMPOUNDS -- PART OF THE CLASS 532-570...,Quinoline or isoquinoline (including hydrogena...,Chemistry,Biotechnology,disclos mc-3/mc-4 receptor ligand ligand follo...
39312,10240186,"Devices, systems, and methods for magnetic sep...",NaN,NaN,Instruments,Analysis of biological materials,method microfluid devic instrument magnet sepa...
39313,5974763,Cell-inside-a-cell honeycomb material,STOCK MATERIAL OR MISCELLANEOUS ARTICLES,Honeycomb type cells extend perpendicularly to...,Mechanical engineering,Textile and paper machines,honeycomb insul panel describ wherein cell pan...


In [5]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# Load pre-trained sentence embeddings model (You need to install sentence-transformers library)
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Function to compute similarity scores between patent abstracts and academic titles
def compute_similarity(patent_abstract, academic_titles):
    # Encode patent abstract and academic titles into embeddings
    patent_embedding = model.encode(patent_abstract)
    academic_embeddings = model.encode(academic_titles)
    
    # Compute cosine similarity between patent abstract and academic titles
    similarities = cosine_similarity([patent_embedding], academic_embeddings)[0]
    return similarities

# Initialize a list to store matched pairs
matched_pairs = []

In [6]:
# Initialize tqdm with the total number of iterations
total_iterations = len(patent)
with tqdm(total=total_iterations, desc="Matching Progress") as pbar:
    # Iterate over patent abstracts
    for index, row in patent.iterrows():
        # Preprocess patent abstract
        preprocessed_abstract = row['processed_abstract']
        
        # Compute similarity scores between preprocessed patent abstract and preprocessed academic titles
        similarities = compute_similarity(preprocessed_abstract, mag['processed_title'])
        
        # Find the index with maximum similarity score
        max_similarity_index = similarities.argmax()
        
        # Check if maximum similarity score is greater than a threshold (e.g., 0.5)
        if similarities[max_similarity_index] > 0.5:
            # Extract matched entity_id
            matched_entity_id = mag.loc[max_similarity_index, 'entity_id']
            
            # Append matched pair to the list
            matched_pairs.append({'entity_id': matched_entity_id, 'patent_id': row['patent_id']})
        else:
            # If no match found above threshold, append None for entity_id
            matched_pairs.append({'entity_id': None, 'patent_id': row['patent_id']})
        
        # Update the progress bar
        pbar.update(1)

Matching Progress:   0%|          | 48/39315 [16:04<219:11:13, 20.10s/it]


KeyboardInterrupt: 

In [ ]:
# Create a DataFrame from matched pairs
matched_pairs_df = pd.DataFrame(matched_pairs)
matched_pairs_df

In [ ]:
# Save the DataFrame to a CSV file
matched_pairs_df.to_csv('matched_pairs.csv', index=False)